<a href="https://colab.research.google.com/github/ksw9092ksw2902/DAB/blob/main/%EC%84%B1%EB%B6%81%EA%B5%ACRF_%EC%88%98%EC%A0%95%EB%B2%84%EC%A0%84(%EC%9E%A0%EC%A0%95%EC%B5%9C%EC%A2%85)%20%EA%B6%8C%EC%88%9C%EC%9A%B0%20%EC%88%98%EC%A0%95%EB%B2%84%EC%A0%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# =============================
# 1️⃣ Google Drive 연결
# =============================
'''
from google.colab import drive
drive.mount('/content/drive')
'''
# =============================
# 2️⃣ 라이브러리 설치 및 불러오기
# =============================
!pip install chardet openpyxl holidays
!pip install pandas numpy chardet openpyxl holidays scikit-learn matplotlib

import pandas as pd
import numpy as np
import chardet, gc
import holidays
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error



# =============================
# 4️⃣ 데이터 불러오기
# =============================
seongbuk_24 = pd.read_excel("C:/Users/eilee/OneDrive/바탕 화면/ITS/DAB/RF/df_seongbuk_rent_24.xlsx")
seongbuk_23 = pd.read_csv("C:/Users/eilee/OneDrive/바탕 화면/ITS/DAB/RF/df_seongbuk_rent_23_edit.csv",encoding='utf-8')
seongbuk_22 = pd.read_csv("C:/Users/eilee/OneDrive/바탕 화면/ITS/DAB/RF/df_seongbuk_rent_22_edit.csv",encoding='utf-8', dtype={'5': object})

# 정렬
for df in [seongbuk_24, seongbuk_23, seongbuk_22]:
    df.sort_values(by='대여일시', inplace=True)

# 24년 필요한 칼럼만 남기기
seongbuk_24_input = seongbuk_24.drop(['이용시간(분)', '이용거리(M)', '생년', '대여대여소ID', '반납대여소ID'], axis=1)
seongbuk_23_input = seongbuk_23.drop(['자전거번호'], axis=1)
seongbuk_22_input = seongbuk_22.drop(['자전거번호'], axis=1)

# datetime 변환
for df in [seongbuk_24_input, seongbuk_23_input, seongbuk_22_input]:
    df['대여일시'] = pd.to_datetime(df['대여일시'])

# =============================
# 5️⃣ 주중/주말/공휴일 칼럼 추가
# =============================
kr_holidays = holidays.KR(years=[2022,2023,2024])

def get_day_type(date):
    if date in kr_holidays:
        return '공휴일'
    elif date.weekday() >= 5:
        return '주말'
    else:
        return '주중'

for df in [seongbuk_24_input, seongbuk_23_input, seongbuk_22_input]:
    df['주중/주말/공휴일'] = df['대여일시'].apply(get_day_type)

# =============================
# 6️⃣ 기상데이터 불러오기 및 정리
# =============================
weather_24 = pd.read_csv("C:/Users/eilee/OneDrive/바탕 화면/ITS/DAB/RF/24년 시간대별 기상정보.csv", encoding='cp949')
weather_23 = pd.read_csv("C:/Users/eilee/OneDrive/바탕 화면/ITS/DAB/RF/23년 시간대별 기상정보.csv", encoding='cp949')
weather_22 = pd.read_csv("C:/Users/eilee/OneDrive/바탕 화면/ITS/DAB/RF/22년 시간대별 기상정보.csv", encoding='cp949')

def clean_weather(df):
    df = df[['일시', '기온(°C)', '강수량(mm)']].copy()
    df['일시'] = pd.to_datetime(df['일시'])
    df['강수량(mm)'] = df['강수량(mm)'].fillna(0)
    return df

weather_22_input = clean_weather(weather_22)
weather_23_input = clean_weather(weather_23)
weather_24_input = clean_weather(weather_24)

del weather_22, weather_23, weather_24
gc.collect()

# =============================
# 7️⃣ 대여 데이터 병합 준비
# =============================
rent_seongbuk_22_input = seongbuk_22_input.drop(['반납일시','반납대여소번호','반납대여소명'], axis=1)
rent_seongbuk_23_input = seongbuk_23_input.drop(['반납일시','반납대여소번호','반납대여소명'], axis=1)
rent_seongbuk_24_input = seongbuk_24_input.drop(['반납일시','반납대여소번호','반납대여소명'], axis=1)

all_rentals = pd.concat([rent_seongbuk_22_input, rent_seongbuk_23_input, rent_seongbuk_24_input])

# 3시간 단위 시간대 구분
def get_hour_bin(datetime_obj):
    bins = ['00-03','03-06','06-09','09-12','12-15','15-18','18-21','21-24']
    idx = datetime_obj.hour // 3
    return bins[idx]

all_rentals['대여시간대_3시간'] = all_rentals['대여일시'].apply(get_hour_bin)

rentals_agg = all_rentals.groupby([
    all_rentals['대여일시'].dt.date,
    '대여시간대_3시간',
    '대여 대여소번호',
    '대여 대여소명',
    '주중/주말/공휴일'
]).size().reset_index(name='대여량')

# =============================
# 8️⃣ 기상데이터 집계
# =============================
all_weather = pd.concat([weather_22_input, weather_23_input, weather_24_input])
all_weather['시간대_3시간'] = all_weather['일시'].apply(get_hour_bin)
weather_agg = all_weather.groupby([
    all_weather['일시'].dt.date,
    '시간대_3시간'
]).agg(
    평균기온=('기온(°C)','mean'),
    총강수량=('강수량(mm)','sum')
).reset_index()

# =============================
# 9️⃣ 병합
# =============================
rentals_agg['날짜'] = pd.to_datetime(rentals_agg['대여일시'])
weather_agg['날짜'] = pd.to_datetime(weather_agg['일시'])

merged_df = pd.merge(
    rentals_agg, weather_agg,
    left_on=['날짜','대여시간대_3시간'],
    right_on=['날짜','시간대_3시간'],
    how='left'
).drop('시간대_3시간', axis=1)

# 날짜 관련 변수 추가
merged_df['년'] = merged_df['날짜'].dt.year
merged_df['월'] = merged_df['날짜'].dt.month
merged_df['일'] = merged_df['날짜'].dt.day
merged_df['요일'] = merged_df['날짜'].dt.dayofweek

# =============================
# 🔟 2025년 예측용 데이터 생성
# =============================
from itertools import product
dates_2025 = pd.date_range(start='2025-01-01', end='2025-12-31', freq='D')
unique_stations = merged_df['대여 대여소번호'].unique()
unique_time_bins = ['00-03','03-06','06-09','09-12','12-15','15-18','18-21','21-24']

prediction_data_2025 = pd.DataFrame(list(product(dates_2025, unique_stations, unique_time_bins)),
                                    columns=['날짜','대여 대여소번호','대여시간대_3시간'])
prediction_data_2025['년'] = prediction_data_2025['날짜'].dt.year
prediction_data_2025['월'] = prediction_data_2025['날짜'].dt.month
prediction_data_2025['일'] = prediction_data_2025['날짜'].dt.day
prediction_data_2025['요일'] = prediction_data_2025['날짜'].dt.dayofweek

kr_holidays_2025 = holidays.KR(years=2025)
prediction_data_2025['주중/주말/공휴일'] = prediction_data_2025['날짜'].apply(get_day_type)

# 2025 날씨는 과거 평균값으로 대체
weather_avg_historical = weather_agg.groupby(['시간대_3시간']).agg(
    평균기온=('평균기온','mean'),
    총강수량=('총강수량','mean')
).reset_index()

prediction_data_2025 = pd.merge(
    prediction_data_2025, weather_avg_historical,
    left_on='대여시간대_3시간', right_on='시간대_3시간', how='left'
).drop('시간대_3시간', axis=1)

# =============================
# 11️⃣ 학습/예측용 데이터 준비
# =============================
train_df = merged_df[merged_df['년'] < 2025].copy()
predict_df = prediction_data_2025.copy()

features = ['대여 대여소번호','대여시간대_3시간','주중/주말/공휴일','년','월','일','요일','평균기온','총강수량']
target = '대여량'

X_train = train_df[features].copy()
y_train = train_df[target].astype(np.float32)
X_predict = predict_df[features].copy()

# 범주형은 category로 변환 + LabelEncoding (메모리 절감)
encoders = {}
for col in ['대여 대여소번호','대여시간대_3시간','주중/주말/공휴일']:
    le = LabelEncoder()
    X_train[col] = le.fit_transform(X_train[col])
    X_predict[col] = le.transform(X_predict[col])
    encoders[col] = le

display(train_df.head())
display(predict_df.head())

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/eilee/OneDrive/바탕 화면/ITS/DAB/RF/df_seongbuk_rent_24.xlsx'

In [ ]:
# =============================
# 12️⃣ RandomForest 학습 (성능 + 메모리 밸런스)
# =============================
rf_model = RandomForestRegressor(
    n_estimators=600,      # 트리 수 늘림
    max_depth=None,        # 깊이 제한 해제 (정확도↑, RAM↑)
    max_features=None,     # 전체 피처 사용
    min_samples_split=2,   # 기본값 복원
    n_jobs=-1,
    random_state=42
)
rf_model.fit(X_train, y_train)
print("✅ 랜덤포레스트 학습 완료")

import gc
gc.collect()
# =============================
# 13️⃣ 예측 (청크 처리)
# =============================
chunk_size = 100000
predictions_2025 = []
for i in range(0, len(X_predict), chunk_size):
    preds = rf_model.predict(X_predict[i:i+chunk_size])
    predictions_2025.extend(preds)
    print(f"Processed chunk {i//chunk_size + 1}/{len(X_predict)//chunk_size + 1}")
    gc.collect()

predict_df['예측대여량'] = predictions_2025

# =============================
# 14️⃣ 성능평가
# =============================
y_train_pred = rf_model.predict(X_train)
mae = mean_absolute_error(y_train, y_train_pred)
rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
mape = np.mean(np.abs((y_train - y_train_pred)/(y_train+1e-8))) * 100

print(f"📊 MAE: {mae:.2f}, RMSE: {rmse:.2f}, MAPE: {mape:.2f}%")




In [ ]:
# =============================
# 15️⃣ 결과 저장
# =============================
station_names = merged_df[['대여 대여소번호','대여 대여소명']].drop_duplicates()
final_prediction_df = predict_df[['날짜','대여시간대_3시간','대여 대여소번호','예측대여량']].merge(
    station_names, on='대여 대여소번호', how='left'
)[['날짜','대여시간대_3시간','대여 대여소번호','대여 대여소명','예측대여량']]

final_prediction_df.to_csv('C:/Users/eilee/OneDrive/바탕 화면/2025_따릉이_대여량_예측_RF최적화.csv', index=False, encoding='utf-8-sig')
print("💾 '/content/drive/MyDrive/2025_따릉이_대여량_예측_RF최적화.csv' 저장 완료")